In [ ]:
import math

def create_distance(temp1, temp2) :
    temp_ = temp1 - temp2
    length = len(temp_)
    distance = 0
    
    for i in range(0, length) :
        distance = distance + pow(temp_[i], 2)
        
    distance = math.sqrt(distance)
    
    return distance

def slopee(x1, y1, x2, y2):
    x = (y2 - y1) / (x2 - x1)
    
    return x

def func(a, b, c) :
    if (b * b - 4 * a * c) >= 0 :
        flag = 1
        a1 = (0 - b + math.sqrt(b * b - 4 * a * c)) / (2 * a)
        a2 = (0 - b - math.sqrt(b * b - 4 * a * c)) / (2 * a)
    else :
        return 0, 0, 0
    
    if a1 < a2 :
        return 1, a1, a2
    else :
        return 1, a2, a1
    
def scale_minmax(col) :
    return (col - col.min()) / (col.max() - col.min())

In [1]:
def get_BC(a, b, gap_nums) :
    if (len(a) == 0) | (len(b) == 0) :
        return 0
    
    temp_min = np.array([a.min(), a.max(), b.min(), b.max()]).min()
    temp_max = np.array([a.min(), a.max(), b.min(), b.max()]).max()
    
    gap = (temp_max - temp_min) / (gap_nums - 1)
    
    if gap == 0 :
        return 1
    
    temp_for_BC = pd.DataFrame(np.arange(3.0).reshape(1, 3), columns = ['gap', 'Class', 'frequency'])
    temp_for_BC['gap'][0] = 0
    temp_for_BC['Class'][0] = 'null'
    temp_for_BC['frequency'][0] = 0.0

    for i in range(0, gap_nums) :
        temp_for_BC.loc[len(temp_for_BC.index)] = [i, 'a', 0.0]
        temp_for_BC.loc[len(temp_for_BC.index)] = [i, 'b', 0.0]

    temp_for_BC = temp_for_BC[1:]
    temp_for_BC = temp_for_BC.reset_index().drop('index', axis = 1)
    
    for i in range(0, len(a)) :
        temp_value = int((a[i] - temp_min) / gap)
        temp_index = temp_for_BC[(temp_for_BC['Class'] == 'a') &
                                 (temp_for_BC['gap'] == temp_value)].index
        temp_for_BC['frequency'][temp_index] += 1

    for i in range(0, len(b)) :
        temp_value = int((b[i] - temp_min) / gap)
        temp_index = temp_for_BC[(temp_for_BC['Class'] == 'b') &
                                 (temp_for_BC['gap'] == temp_value)].index
        temp_for_BC['frequency'][temp_index] += 1
    
    BC = 0
    loc = 0

    for i in range(0, temp_for_BC['gap'].nunique()) :
        temp_a_frequency = (temp_for_BC[(temp_for_BC['gap'] == i) & (temp_for_BC['Class'] == 'a')]['frequency'] / len(a))[loc]
        loc += 1
        temp_b_frequency = (temp_for_BC[(temp_for_BC['gap'] == i) & (temp_for_BC['Class'] == 'b')]['frequency'] / len(b))[loc]
        loc += 1

        temp_value = np.sqrt(temp_a_frequency * temp_b_frequency)
        BC = BC + temp_value
        
    return BC

In [ ]:
def get_BC2(df_Class0, df_Class1, gap_nums) :
    BC = 0
        
    for f in range(0, len(features)) :
        a_feature = np.array(df_Class0[features[f]])
        b_feature = np.array(df_Class1[features[f]])
        BC = BC + get_BC(a_feature, b_feature, gap_nums)
        
    return BC

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

def DP_weighting(k, k2, df, df_Class1, gap_nums) :
    DP_sample_weight = pd.DataFrame(np.arange(3.0).reshape(1, 3), 
                                    columns = ['index_Class1', 'same_samples', 'BC'])
    DP_sample_weight['index_Class1'][0] = 'null' 
    DP_sample_weight['same_samples'][0] = 0.0
    DP_sample_weight['BC'][0] = 0.0 
    DP_sample_weight = DP_sample_weight[1:]
    
    KNN = KNeighborsClassifier(n_neighbors = k)
    KNN_x_train = np.array(df[features])
    KNN_y_train = np.array(df.loc[:, 'Class']).reshape(-1, 1)
    KNN.fit(KNN_x_train, KNN_y_train)
    
    for i in df_Class1.index :
        temp_index = df_Class1['index'][i]
        temp_index_forBC = df[df['index'] == temp_index].index[0]
        temp_index_forBC2 = KNN.kneighbors(KNN_x_train[temp_index_forBC].reshape(1, len(features)), 
                                           k2 + 1, False)[0]
        temp_df_forBC = df.iloc[temp_index_forBC2, :]
            
        temp_same_samples = temp_df_forBC[temp_df_forBC['Class'] == 1].shape[0] - 1
        temp_same_samples_ = -1 * np.power((temp_same_samples / k2 - 0.5), 2) + 0.25
        
        temp_data_Class0 = temp_df_forBC[temp_df_forBC['Class'] == 0][features]
        temp_data_Class1 = temp_df_forBC[temp_df_forBC['Class'] == 1][features]
        temp_BC = get_BC2(temp_data_Class0, temp_data_Class1, gap_nums)
        DP_sample_weight.loc[len(DP_sample_weight.index)] = [temp_index, temp_same_samples_, temp_BC]
        
    
    DP_sample_weight['BC'] = scale_minmax(DP_sample_weight['BC'])
    DP_sample_weight['BC2'] = 1 - DP_sample_weight['BC']
    
    DP_sample_weight['weight'] = DP_sample_weight['same_samples'] * DP_sample_weight['BC2']
    DP_sample_weight['weight'] = scale_minmax(DP_sample_weight['weight'])
    
    return DP_sample_weight

In [ ]:
def create_new_sample1(f, center, temp_X, Dis_value) :
    k_o = slopee(center[0], center[f], temp_X[0], temp_X[f])
    k_r = -1 / k_o
    b = center[f] - k_r * center[0]
        
    A = k_r * k_r + 1
    B = 2 * k_r * (b - center[f]) - 2 * center[0]
    C = np.power(center[0], 2) + np.power((b - center[f]), 2) - np.power(Dis_value, 2)
        
    D1_x = func(A, B, C)[1]
    D1_y = k_r * D1_x + b
    D2_x = func(A, B, C)[2]
    D2_y = k_r * D2_x + b
    
    A2 = k_r * k_r + 1
    B2 = 2 * k_r * (b - center[f]) - 2 * center[0]
    C2 = np.power(center[0], 2) + np.power((b - center[f]), 2) - np.power(2 * Dis_value, 2)

    k1_x = func(A2, B2, C2)[1]
    k1_y = k_r * k1_x + b
    k2_x = func(A2, B2, C2)[2]
    k2_y = k_r * k2_x + b
    
    return k_o, k_r, b, D1_x, D1_y, D2_x, D2_y, k1_x, k1_y, k2_x, k2_y

def create_new_sample2(center, temp_X, weight_) :
    for f in range(0, len(features)) :
        if center[f] == temp_X[f] :
            temp_new = temp_X + random.random() * (center - temp_X)
            return 1, temp_new
        
    temp_border = center - (temp_X - center)
    
    temp_new_f1_min = center[0] - 2 * np.abs(temp_X[0] - center[0]) * weight_
    temp_new_f1_max = center[0] + 2 * np.abs(temp_X[0] - center[0]) * weight_
    temp_new_f1 = random.uniform(temp_new_f1_min, temp_new_f1_max)
        
    if (temp_new_f1 < data_scaler[features[0]].min()) | (temp_new_f1 > data_scaler[features[0]].max()) :
        return 0, np.array([0])
    
    temp_new = np.array(temp_new_f1)
    
    for f in range(1, len(features)) :
        Dis_value = create_distance(np.array((center[0], center[f])), 
                                    np.array((temp_X[0], temp_X[f]))) * weight_
        k_o, k_r, b, D1_x, D1_y, D2_x, D2_y, k1_x, k1_y, k2_x, k2_y = create_new_sample1(f, center, temp_X, Dis_value)
        
        a_1 = np.power(temp_new_f1 - center[0], 2)
        b_1 = 1 + np.power(k_o, 2)
        c_1 = temp_new_f1 - k1_x
        d_1 = k1_y + k_o * c_1
        e_1 = center[f]

        A1 = b_1 - 1
        B1 = 2 * (d_1 - b_1 * e_1)
        C1 = a_1 * b_1 + b_1 * np.power(e_1, 2) - np.power(d_1, 2)
        
        temp_value1_flag = func(A1, B1, C1)[0]
        if temp_value1_flag == 1 :
            temp_value1_1 = func(A1, B1, C1)[1]
            temp_value1_2 = func(A1, B1, C1)[2]
        else :
            return 0, np.array([0])
    
        if (create_distance(np.array([center[0], center[f]]), np.array([temp_new_f1, temp_value1_1])) < 
            create_distance(np.array([center[0], center[f]]), np.array([temp_new_f1, temp_value1_2]))) :
            temp_value1 = temp_value1_1
        else :
            temp_value1 = temp_value1_2
            
        a_2 = np.power((temp_new_f1 - center[0]), 2)
        b_2 = 1 + np.power(k_o, 2)
        c_2 = temp_new_f1 - k2_x
        d_2 = k2_y + k_o * c_2
        e_2 = center[f]
        
        A2 = b_2 - 1
        B2 = 2 * (d_2 - b_2 * e_2)
        C2 = a_2 * b_2 + b_2 * np.power(e_2, 2) - np.power(d_2, 2)
        
        temp_value2_flag = func(A2, B2, C2)[0]
        if temp_value2_flag == 1 :
            temp_value2_1 = func(A2, B2, C2)[1]
            temp_value2_2 = func(A2, B2, C2)[2]
        else :
            return 0, np.array([0])
        
        if (create_distance(np.array([center[0], center[f]]), np.array([temp_new_f1, temp_value2_1])) < 
            create_distance(np.array([center[0], center[f]]), np.array([temp_new_f1, temp_value2_2]))) :
            temp_value2 = temp_value2_1
        else :
            temp_value2 = temp_value2_2
            
        temp_new_f = random.uniform(temp_value2, temp_value1)

        if (temp_new_f < data_scaler[features[f]].min()) | (temp_new_f > data_scaler[features[f]].max()) :
            return 0, np.array([0])
        
        temp_new = np.append(temp_new, temp_new_f)
        
    return 1, temp_new

In [ ]:
def DP_cs(k, k2, gap_nums, new_sample_nums, df, df_Class1_o, new_data) :
    print('本轮需生成', new_sample_nums, '个点')
    df_Class1 = df_Class1_o.copy()
    
    count = 0
    
    temp_new_data_pool = []
    
    # 第一列 index；第二列 生成非法样本的次数；第三列 缩短权值的次数
    L = np.zeros((df_Class1.shape[0], 3))
    
    for i in range(0, df_Class1.shape[0]) :
        L[i][0] = df_Class1['index'][i]
    
    KNN2 = KNeighborsClassifier(n_neighbors = k)
    KNN_x_train2 = np.array(df_Class1[features])
    KNN_y_train2 = np.array(df_Class1.loc[:, 'Class']).reshape(-1, 1)
    KNN2.fit(KNN_x_train2, KNN_y_train2)
    
    while count < new_sample_nums :
        if df_Class1.shape[0] <= k + 1:
            break
            
        temp = df_Class1.sample(n = 1, replace = True, random_state = None, axis = 0)
        temp_X = np.array(temp[features])[0]
        temp_index = int(temp['index'])
        temp_weight = float(temp['weight']) / np.power(1.05, L[temp.index[0]][2])
        
        temp_weight_ = 0.25 * (temp_weight + 1)
        
        temp_Xj_index = KNN2.kneighbors(KNN_x_train2[temp.index[0]].reshape(1, len(features)), 
                                        k + 1, False)[0][1 :]
        temp_df_forXj = df_Class1.iloc[temp_Xj_index, :]
        
        temp_df_forXj = temp_df_forXj[temp_df_forXj['weight'] >= temp_weight]
        if temp_df_forXj.shape[0] == 0 :
            continue
        
        temp_df_forXj['distance'] = 0
        for i in range(0, temp_df_forXj.shape[0]) :
            temp_df_forXj['distance'][temp_df_forXj.index[i]] = create_distance(temp_X, 
                                                                                np.array(temp_df_forXj.loc[temp_df_forXj.index[i], features]))
        temp_std = temp_df_forXj['distance'].std()
        
        while temp_std >= 0.04 :
            if temp_df_forXj.shape[0] <= 1 :
                break
            temp_df_forXj = temp_df_forXj.iloc[0 : -1, :]
            temp_std = temp_df_forXj['distance'].std()
            
        temp_Xj_o = temp_df_forXj.sample(n = 1, replace = True, random_state = None, axis = 0)
        temp_Xj= np.array(temp_Xj_o[features])[0]
        
        center = (temp_X + temp_Xj) / 2
        
        temp_new_flag, temp_new = create_new_sample2(center, temp_X, temp_weight_)
        
        if temp_new_flag == 0 :
            continue
            
        temp_new_ = np.array(-100)
        temp_new_ = np.append(temp_new_, temp_new)
        temp_new_ = np.append(temp_new_, 1)
        
        df_forBC = df.copy()
        df_forBC.loc[len(df_forBC.index)] = temp_new_
        
        KNN = KNeighborsClassifier(n_neighbors = k2)
        KNN_x_train = np.array(df_forBC[features])
        KNN_y_train = np.array(df_forBC.loc[:, 'Class']).reshape(-1, 1)
        KNN.fit(KNN_x_train, KNN_y_train)
        
        temp_index_forBC = KNN.kneighbors(KNN_x_train[-1].reshape(1, len(features)), k2 + 1, False)[0]
        temp_df_forBC = df_forBC.iloc[temp_index_forBC[1 :], :]
        temp_df_forBC2 = df_forBC.iloc[temp_index_forBC, :]
        
        if temp_df_forBC[temp_df_forBC['Class'] == 0].shape[0] == k2 :
            print('新样本周围无同类样本')
            continue
        
        temp_data_Class0 = temp_df_forBC[temp_df_forBC['Class'] == 0][features]
        temp_data_Class1 = temp_df_forBC[temp_df_forBC['Class'] == 1][features]
        temp_data_Class0_ = temp_df_forBC2[temp_df_forBC2['Class'] == 0][features]
        temp_data_Class1_ = temp_df_forBC2[temp_df_forBC2['Class'] == 1][features]

        origin_BC = 0
        
        for f in range(0, len(features)) :
            a_feature = np.array(temp_data_Class0[features[f]])
            b_feature = np.array(temp_data_Class1[features[f]])
            origin_BC = origin_BC + get_BC(a_feature, b_feature, gap_nums)
            
        new_BC = 0
        
        for f in range(0, len(features)) :
            a_feature = np.array(temp_data_Class0_[features[f]])
            b_feature = np.array(temp_data_Class1_[features[f]])
            new_BC = new_BC + get_BC(a_feature, b_feature, gap_nums)
        
        if new_BC <= origin_BC :
            new_data.append(temp_new)
            temp_new_data_pool.append(temp_new)
            count += 1
            if count % 200 == 0 :
                print(count, 'is done')
        
        if L[temp.index[0]][1] < 19 :
            L[temp.index[0]][1] += 1
            continue
        
        if L[temp.index[0]][2] < 2 :
            print(temp_index, '号样本累计生成不合法的样本点超过 20个，减小权重')
            L[temp.index[0]][1] = 0
            L[temp.index[0]][2] += 1
        else :
            print(temp_index, '号样本累计减小权重 3次')
            df_Class1 = df_Class1.drop(temp.index[0], axis = 0)
            df_Class1 = df_Class1.reset_index()
            df_Class1 = df_Class1.drop('level_0', axis = 1)
            
            KNN2 = KNeighborsClassifier(n_neighbors = k)
            KNN_x_train2 = np.array(df_Class1[features])
            KNN_y_train2 = np.array(df_Class1.loc[:, 'Class']).reshape(-1, 1)
            KNN2.fit(KNN_x_train2, KNN_y_train2)
            
    temp_new_data_pool = pd.DataFrame(temp_new_data_pool)
            
    for i in range(0, new_sample_nums - count) :
        temp_new = np.array(temp_new_data_pool.sample(n = 1, replace = True))[0]
        new_data.append(temp_new)
        
    print('生成完毕')